In [1]:
# !pip install -r requirements.txt

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import logging
import multiprocessing
from datetime import datetime, timedelta

from src import modules as f
%load_ext autoreload
%autoreload 2

In [4]:
# Download, load, extract, model...
interval = '1d'
stock_set = f.download_interval_process(interval, 1)
print(f'Stock set: \n{stock_set}\n')

Stock set: 
{'DUK', 'EXC', 'TMUS', 'DIS', 'CRM', 'OKE', 'WFC', 'COP', 'UNH', 'IBM', 'PLTR', 'XOM', 'GS', 'AJG', 'TDG', 'MA', 'DKNG', 'CSCO', 'AMGN', 'SRE', 'SPGI', 'MSFT', 'NFLX', 'AMZN', 'WSM', 'WMB', 'LLY', 'JNJ', 'EXE', 'ATO', 'JPM', 'SW', 'PM', 'TTWO', 'D', 'MDLZ', 'MO', 'CL', 'GOOGL', 'XEL', 'DECK', 'ABBV', 'LNG', '^VIX', 'HD', 'NVDA', 'SLB', 'AXP', 'EOG', 'WMT', 'COST', 'ADBE', 'TGT', 'TPR', 'DASH', 'KMI', 'ACN', 'T', 'FCNCA', 'TSLA', 'CVX', 'TMO', 'MRK', 'BRK-B', 'BSX', 'VST', 'PGR', 'CMCSA', 'APH', 'PEP', 'NRG', 'VZ', 'FWONK', 'PG', 'PSX', 'WELL', 'META', 'AEP', 'CEG', 'ORCL', 'ISRG', 'BAC', 'NEE', 'RCL', 'GOOG', 'MSI', 'KO', 'SO', 'AVGO', 'AAPL', 'ABT', 'V'}



In [ ]:
f.make_table_features_process(stock_set, interval, 8)

In [ ]:
f.make_master_table(stock_set, interval)

In [ ]:
f.xg_boost_model(interval='1d', grid_search_on=True)

In [11]:
symbol = 'TSLA'.upper()
interval = '1d'
f.model_prospect(symbol, interval, build=False)
df_val = f.model_validation(symbol, interval)
correctly_predicted = df_val.dir_pred_match.sum()
print(f'\nCorrectly Predicted: {correctly_predicted}')
entries_predicted = df_val.shape[0]
print(f'Entries Predicted: {entries_predicted}')
print(f'Percent Correct: {correctly_predicted / entries_predicted}')


Model Prediction 1d TSLA:
Predicted Next 1d Movement: UP

Model Prediction Probabilities:
no_change (0): 0.3398
up (1): 0.3494
down (2): 0.3108

Last Entry 1d TSLA Datetime Used for Prediction:
NOTE: It's in or contains the full 1d time interval.
EST: 2025-04-04 12:00:00 AM EDT-0400
PDT: 2025-04-03 09:00:00 PM PDT-0700

Correctly Predicted: 13
Entries Predicted: 50
Percent Correct: 0.26
